In [1]:
#needed after restructuring of directory
import os
os.chdir('/Users/alexchandler/UFC_Prediction_2022')
#getting dependencies
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn' (disables SettingWithCopyWarning)

import numpy as npy
from datetime import datetime
from datetime import date
import matplotlib.pyplot as plt
import random
import sklearn
import scipy
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import VarianceThreshold
import itertools
#this imports all of the functions from the file functions.py
os.chdir('/Users/alexchandler/UFC_Prediction_2022/src')
from functions import *
os.chdir('/Users/alexchandler/UFC_Prediction_2022')

#importing csv fight data and saving as dataframes
ufc_fights_winner = pd.read_csv('src/models/buildingMLModel/data/processed/ufc_fights.csv',low_memory=False)
ufc_fights_method = pd.read_csv('src/models/buildingMLModel/data/processed/ufc_fights.csv',low_memory=False)
ufcfighterscrap =pd.read_csv('src/models/buildingMLModel/data/processed/fighter_stats.csv',sep=',',low_memory=False)

In [2]:
#cleaning the methods column for winner prediction 
#changing anything other than 'U-DEC','M-DEC', 'KO/TKO', 'SUB', to 'bullshit'
#changing 'U-DEC','M-DEC', to 'DEC'
#(counting split decisions as bullshit)
ufc_fights_winner['method'] = clean_method_for_winner_vect(ufc_fights_winner['method'])
ufc_fights_winner['method'].unique()

#cleaning the methods column for method prediction 
#changing anything other than 'U-DEC','M-DEC', 'S-DEC', 'KO/TKO', 'SUB', to 'bullshit'
#changing 'U-DEC','M-DEC', 'S-DEC', to 'DEC'
#(counting split decisions as decisions)
ufc_fights_method['method'] = clean_method_vect(ufc_fights_method['method'])
ufc_fights_method['method'].unique()

#getting rid of rows with incomplete or useless data
#fights with outcome "Win" or "Loss" (no "Draw")
draw_mask=ufc_fights_winner['result'] != 'D' 
#fights where the method of victory is TKO/SUB/DEC (no split decision or DQ or Overturned or anything else like that)
method_mask_winner=(ufc_fights_winner['method']!='bullshit')
method_mask_method=(ufc_fights_method['method']!='bullshit')
#fights where age is known
age_mask=(ufc_fights_winner['fighter_age']!='unknown')&(ufc_fights_winner['opponent_age']!='unknown')
#fights where height reach is known
height_mask=(ufc_fights_winner['fighter_height']!='unknown')&(ufc_fights_winner['opponent_height']!='unknown')
reach_mask=(ufc_fights_winner['fighter_reach']!='unknown')&(ufc_fights_winner['opponent_reach']!='unknown')
#fights where number of wins is known
wins_mask=(ufc_fights_winner['fighter_wins'] != 'unknown' )& (ufc_fights_winner['opponent_wins'] != 'unknown')
#fights where both fighters have strike statistics (gets rid of UFC debuts)
strikes_mask=(ufc_fights_winner['fighter_inf_sig_strikes_attempts_avg'] != 0)&(ufc_fights_winner['opponent_inf_sig_strikes_attempts_avg'] != 0)
#includes only the fights satisfying these conditions
ufc_fights_winner=ufc_fights_winner[draw_mask&method_mask_winner&age_mask&height_mask&reach_mask&wins_mask&strikes_mask]
ufc_fights_method=ufc_fights_method[draw_mask&method_mask_method&age_mask&height_mask&reach_mask&wins_mask&strikes_mask]
ufc_fights=ufc_fights[draw_mask&method_mask_winner&age_mask&height_mask&reach_mask&wins_mask&strikes_mask]

#listing all stats and making some new stats from them (differences often score higher in the learning models)
record_statistics=[u'fighter_wins', 
                   u'fighter_losses',
                   u'fighter_L5Y_wins',
                   u'fighter_L5Y_losses', 
                   u'fighter_L2Y_wins', 
                   u'fighter_L2Y_losses',
                    u'fighter_ko_wins',
                   u'fighter_ko_losses',
                   u'fighter_L5Y_ko_wins',
                   u'fighter_L5Y_ko_losses',
                   u'fighter_L2Y_ko_wins',
                    u'fighter_L2Y_ko_losses',
                   u'fighter_sub_wins',
                   u'fighter_sub_losses',
                   u'fighter_L5Y_sub_wins',
                    u'fighter_L5Y_sub_losses', 
                   u'fighter_L2Y_sub_wins', 
                   u'fighter_L2Y_sub_losses',
                   u'opponent_wins', 
                   u'opponent_losses',
                   u'opponent_L5Y_wins', 
                   u'opponent_L5Y_losses', 
                   u'opponent_L2Y_wins', 
                   u'opponent_L2Y_losses', 
                    u'opponent_ko_wins', 
                   u'opponent_ko_losses', 
                   u'opponent_L5Y_ko_wins', 
                   u'opponent_L5Y_ko_losses', 
                   u'opponent_L2Y_ko_wins',
                    u'opponent_L2Y_ko_losses', 
                   u'opponent_sub_wins', 
                   u'opponent_sub_losses',
                   u'opponent_L5Y_sub_wins', 
                    u'opponent_L5Y_sub_losses', 
                   u'opponent_L2Y_sub_wins', 
                   u'opponent_L2Y_sub_losses']

physical_stats=[ u'fighter_age',
                u'fighter_height',
                    u'fighter_reach',  
                u'opponent_age',  
                u'opponent_height',
                    u'opponent_reach']

#THERE MAY BE A PROBLEM IN AGE HEIGHT REACH TO DO WITH STRING VS FLOAT. MAKE SURE THESE ARE ALL THE CORRECT TYPE
#MAYBE WE ARE LOSING PREDICTABILITY HERE

#here is the list of all stats available (besides stance), does not include names or result
punch_statistics=[    u'fighter_inf_knockdowns_avg',
                    u'fighter_inf_pass_avg',
                    u'fighter_inf_reversals_avg',
                    u'fighter_inf_sub_attempts_avg',
                    u'fighter_inf_takedowns_landed_avg',
                    u'fighter_inf_takedowns_attempts_avg',
                    u'fighter_inf_sig_strikes_landed_avg',
                    u'fighter_inf_sig_strikes_attempts_avg',
                    u'fighter_inf_total_strikes_landed_avg',
                    u'fighter_inf_total_strikes_attempts_avg',
                    u'fighter_inf_head_strikes_landed_avg',
                    u'fighter_inf_head_strikes_attempts_avg',
                    u'fighter_inf_body_strikes_landed_avg',
                    u'fighter_inf_body_strikes_attempts_avg',
                    u'fighter_inf_leg_strikes_landed_avg',
                    u'fighter_inf_leg_strikes_attempts_avg',
                    u'fighter_inf_distance_strikes_landed_avg',
                    u'fighter_inf_distance_strikes_attempts_avg',
                    u'fighter_inf_clinch_strikes_landed_avg',
                    u'fighter_inf_clinch_strikes_attempts_avg',
                    u'fighter_inf_ground_strikes_landed_avg',
                    u'fighter_inf_ground_strikes_attempts_avg',
                
                    u'fighter_abs_knockdowns_avg',
                    u'fighter_abs_pass_avg',
                    u'fighter_abs_reversals_avg',
                    u'fighter_abs_sub_attempts_avg',
                    u'fighter_abs_takedowns_landed_avg',
                    u'fighter_abs_takedowns_attempts_avg',
                    u'fighter_abs_sig_strikes_landed_avg',
                    u'fighter_abs_sig_strikes_attempts_avg',
                    u'fighter_abs_total_strikes_landed_avg',
                    u'fighter_abs_total_strikes_attempts_avg',
                    u'fighter_abs_head_strikes_landed_avg',
                    u'fighter_abs_head_strikes_attempts_avg',
                    u'fighter_abs_body_strikes_landed_avg',
                    u'fighter_abs_body_strikes_attempts_avg',
                    u'fighter_abs_leg_strikes_landed_avg',
                    u'fighter_abs_leg_strikes_attempts_avg',
                    u'fighter_abs_distance_strikes_landed_avg',
                    u'fighter_abs_distance_strikes_attempts_avg',
                    u'fighter_abs_clinch_strikes_landed_avg',
                    u'fighter_abs_clinch_strikes_attempts_avg',
                    u'fighter_abs_ground_strikes_landed_avg',
                    u'fighter_abs_ground_strikes_attempts_avg',
                    
                    u'opponent_inf_knockdowns_avg',
                    u'opponent_inf_pass_avg',
                    u'opponent_inf_reversals_avg',
                    u'opponent_inf_sub_attempts_avg',
                    u'opponent_inf_takedowns_landed_avg',
                    u'opponent_inf_takedowns_attempts_avg',
                    u'opponent_inf_sig_strikes_landed_avg',
                    u'opponent_inf_sig_strikes_attempts_avg',
                    u'opponent_inf_total_strikes_landed_avg',
                    u'opponent_inf_total_strikes_attempts_avg',
                    u'opponent_inf_head_strikes_landed_avg',
                    u'opponent_inf_head_strikes_attempts_avg',
                    u'opponent_inf_body_strikes_landed_avg',
                    u'opponent_inf_body_strikes_attempts_avg',
                    u'opponent_inf_leg_strikes_landed_avg',
                    u'opponent_inf_leg_strikes_attempts_avg',
                    u'opponent_inf_distance_strikes_landed_avg',
                    u'opponent_inf_distance_strikes_attempts_avg',
                    u'opponent_inf_clinch_strikes_landed_avg',
                    u'opponent_inf_clinch_strikes_attempts_avg',
                    u'opponent_inf_ground_strikes_landed_avg',
                    u'opponent_inf_ground_strikes_attempts_avg',
                
                    u'opponent_abs_knockdowns_avg',
                    u'opponent_abs_pass_avg',
                    u'opponent_abs_reversals_avg',
                    u'opponent_abs_sub_attempts_avg',
                    u'opponent_abs_takedowns_landed_avg',
                    u'opponent_abs_takedowns_attempts_avg',
                    u'opponent_abs_sig_strikes_landed_avg',
                    u'opponent_abs_sig_strikes_attempts_avg',
                    u'opponent_abs_total_strikes_landed_avg',
                    u'opponent_abs_total_strikes_attempts_avg',
                    u'opponent_abs_head_strikes_landed_avg',
                    u'opponent_abs_head_strikes_attempts_avg',
                    u'opponent_abs_body_strikes_landed_avg',
                    u'opponent_abs_body_strikes_attempts_avg',
                    u'opponent_abs_leg_strikes_landed_avg',
                    u'opponent_abs_leg_strikes_attempts_avg',
                    u'opponent_abs_distance_strikes_landed_avg',
                    u'opponent_abs_distance_strikes_attempts_avg',
                    u'opponent_abs_clinch_strikes_landed_avg',
                    u'opponent_abs_clinch_strikes_attempts_avg',
                    u'opponent_abs_ground_strikes_landed_avg',
                    u'opponent_abs_ground_strikes_attempts_avg']



#adding record differences to ufc_fights
record_statistics_diff = []
half_length=int(len(record_statistics)/2)
for i in range(half_length):
    ufc_fights_winner[record_statistics[i]+'_diff_2']=ufc_fights_winner[record_statistics[i]]-ufc_fights_winner[record_statistics[i+half_length]]
    ufc_fights_method[record_statistics[i]+'_diff_2']=ufc_fights_method[record_statistics[i]]-ufc_fights_method[record_statistics[i+half_length]]
    record_statistics_diff.append(record_statistics[i]+'_diff_2')
    
#lets try and improve the greedy algorithm by considering differences. Lets start by replacing height and reach by their differences
ufc_fights_winner['height_diff']=ufc_fights_winner['fighter_height'].apply(float)-ufc_fights_winner['opponent_height'].apply(float)
ufc_fights_winner['reach_diff']=ufc_fights_winner['fighter_reach'].apply(float)-ufc_fights_winner['opponent_reach'].apply(float)
ufc_fights_method['height_diff']=ufc_fights_method['fighter_height'].apply(float)-ufc_fights_method['opponent_height'].apply(float)
ufc_fights_method['reach_diff']=ufc_fights_method['fighter_reach'].apply(float)-ufc_fights_method['opponent_reach'].apply(float)


physical_stats_diff = ['fighter_age_diff', 'height_diff', 'reach_diff']

#adding punch differences to ufc_fights
punch_statistics_diff = []
half_length=int(len(punch_statistics)/2)
for i in range(half_length):
    ufc_fights_method[punch_statistics[i]+'_diff_2']=ufc_fights_method[punch_statistics[i]]-ufc_fights_method[punch_statistics[i+half_length]]
    ufc_fights_winner[punch_statistics[i]+'_diff_2']=ufc_fights_winner[punch_statistics[i]]-ufc_fights_winner[punch_statistics[i+half_length]]
    punch_statistics_diff.append(punch_statistics[i]+'_diff_2')
    
possible_stats =record_statistics_diff+physical_stats_diff+punch_statistics_diff

#setting age to float
ufc_fights_winner['fighter_age'] = ufc_fights_winner['fighter_age'].apply(float)
ufc_fights_winner['opponent_age'] = ufc_fights_winner['opponent_age'].apply(float)
ufc_fights_method['fighter_age'] = ufc_fights_method['fighter_age'].apply(float)
ufc_fights_method['opponent_age'] = ufc_fights_method['opponent_age'].apply(float)
ufc_fights_winner['fighter_age_diff'] = ufc_fights_winner['fighter_age']-ufc_fights_winner['opponent_age']

ufc_fights_winner['fighter_age_diff_adj'] = (ufc_fights_winner['fighter_age']-23).apply(abs)-(ufc_fights_winner['opponent_age']-23).apply(abs)



In [3]:
#using 4 at a time to run greedy, found this 4 stat combo giving .638 cross val score with fit_intercept = False
current_best = ['fighter_age_diff', 
                '4-fighter_score_diff', 
                'fighter_abs_distance_strikes_attempts_avg_diff_2', 
                'fighter_abs_total_strikes_landed_avg_diff_2']
ufc_fights_df = ufc_fights_winner[current_best]
winPredictionModel=LogisticRegression(solver='lbfgs', max_iter=2000,fit_intercept=False)
X=ufc_fights_df.iloc[0:2080].to_numpy()
y=ufc_fights_winner['result'].iloc[0:2080]
winPredictionModel.fit(X,y)
print('model score: '+str(winPredictionModel.score(X,y)))
print('cross val score: '+str(cross_val_score(winPredictionModel,X,y,cv=3).mean()))

model score: 0.6240384615384615
cross val score: 0.6264532244359333


In [4]:
#I've defined this in such a way to predict what happens when fighter1 in their day1 version fights fighter2
#in their day2 version. Meaning we could compare for example 2014 Tyron Woodley to 2019 Colby Covington
def ufc_prediction_tuple(fighter1,fighter2,day1=date.today(),day2=date.today()):
    return [fighter_score_diff(fighter1,fighter2,day1, 4),
            fighter_score_diff(fighter1,fighter2,day1, 9),
            fighter_score_diff(fighter1,fighter2,day1, 15),
            fight_math_diff(fighter1,fighter2,day1, 1),
            fight_math_diff(fighter1,fighter2,day1, 6),
            L5Y_sub_wins(fighter1,day1)-L5Y_sub_wins(fighter2,day2),
            L5Y_losses(fighter1,day1)-L5Y_losses(fighter2,day2),
            L5Y_ko_losses(fighter1,day1)-L5Y_ko_losses(fighter2,day2),
            fighter_age(fighter1,day1)-fighter_age(fighter2,day2),
            avg_count('total_strikes_landed',fighter1,'abs',day1)-avg_count('total_strikes_landed',fighter2,'abs',day2),
            avg_count('head_strikes_landed',fighter1,'abs',day1)-avg_count('head_strikes_landed',fighter2,'abs',day2),
            avg_count('ground_strikes_landed',fighter1,'inf',day1)-avg_count('ground_strikes_landed',fighter2,'inf',day2),
            avg_count('takedowns_attempts',fighter1,'inf',day1)-avg_count('takedowns_attempts',fighter2,'inf',day2),
            avg_count('head_strikes_landed',fighter1,'inf',day1)-avg_count('head_strikes_landed',fighter2,'inf',day2),
           ]


def ufc_method_prediction_tuple(fighter1,fighter2,day1=date.today(),day2=date.today()):
    return [ko_losses(fighter1,day1)-ko_losses(fighter2,day2),
            sub_losses(fighter1,day1)-sub_losses(fighter2,day2)]

#improvement after adding fight math stats
current_best = ['4-fighter_score_diff',
 '9-fighter_score_diff',
 '15-fighter_score_diff',
 '1-fight_math',
 '6-fight_math',
 'fighter_L5Y_sub_wins_diff_2',
 'fighter_L5Y_losses_diff_2',
 'fighter_L5Y_ko_losses_diff_2',
 'fighter_age_diff',
 'fighter_abs_total_strikes_landed_avg_diff_2',
 'fighter_abs_head_strikes_landed_avg_diff_2',
 'fighter_inf_ground_strikes_landed_avg_diff_2',
 'fighter_inf_takedowns_attempts_avg_diff_2',
 'fighter_inf_head_strikes_landed_avg_diff_2',
 ]

ufc_fights_df = ufc_fights_winner[current_best]
winPredictionModel=LogisticRegression(solver='lbfgs', max_iter=2000, fit_intercept=False)
X=ufc_fights_df.iloc[0:2080].to_numpy()
y=ufc_fights_winner['result'].iloc[0:2080]
winPredictionModel.fit(X,y)
print('model score: '+str(winPredictionModel.score(X,y)))
print('cross val score: '+str(cross_val_score(winPredictionModel,X,y,cv=3).mean()))
print('coefficients'+str(winPredictionModel.coef_))
print('intercept'+str(winPredictionModel.intercept_))

best_stats=['fighter_ko_losses_diff_2', 'fighter_sub_losses_diff_2']
rfc=RandomForestClassifier()
Xr=ufc_fights_method[best_stats].iloc[0:2300].to_numpy()
yr=ufc_fights_method['method'].iloc[0:2300]
rfc.fit(Xr,yr)

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random

#date entered in the form 'Aug 20, 2013'
#predicts what happens if the day1 version of fighter1 fights fighter2 at their day2 version
#by default day1 and day2 are set to today if left unspecified
def ufc_predict(fighter1,fighter2,day1=date.today(), day2=date.today()):
    fighters=[fighter1,fighter2]
    random_guy=fighters[random.choice([0,1])]
    fighter1_fights=(ufcfightscrap['fighter']==fighter1).sum()
    fighter2_fights=(ufcfightscrap['fighter']==fighter2).sum()
    if fighter1_fights==0 and not fighter2_fights==0:
        print('This is '+fighter1+"'s UFC debut... so obviously I do not have enough data to make an educated pick. But since you are DEMANDING an outcome, hmmm... I guess I'll go with "+random_guy+'.')
    elif fighter2_fights==0 and not fighter1_fights==0:
        print('This is '+fighter2+"'s UFC debut... so obviously I do not have enough data to make an educated pick. But since you are DEMANDING an outcome, hmmm... I guess I'll go with "+random_guy+'.') 
    elif fighter1_fights==0 and fighter2_fights==0:
        print('This is both '+fighter1+"'s "+'and '+fighter2+"'s "+"UFC debuts... so obviously I do not have enough data to make an educated pick. But since you are DEMANDING an outcome, hmmm... I guess I'll go with "+random_guy+'.')
    else:
        try:
            print('The winner is...')
            ufc_tuple=ufc_prediction_tuple(fighter1,fighter2,day1,day2)
            ufc_method_tuple=ufc_method_prediction_tuple(fighter1,fighter2,day1,day2)
            value=winPredictionModel.predict([ufc_tuple])[0]
            result=rfc.predict([ufc_method_tuple])[0]
            if fighter1=="Jon Jones" and fighter2=="Jon Jones":
                print('Daniel Cormier')
            elif value=='W':
                print(fighter1+' by '+result)
            else:
                print(fighter2+' by '+result)
            try:
                fighter1stripped = fighter1.replace(" ","")
                fighter2stripped = fighter2.replace(" ","")
                j=random.randrange(1,5)
                img1 = mpimg.imread('images/'+str(j)+fighter1stripped+'.jpg')
                j=random.randrange(1,5)
                img2 = mpimg.imread('images/'+str(j)+fighter2stripped+'.jpg')
                fig, ax = plt.subplots(1, 2, figsize=(10,10))
                ax[0].set(title=fighter1, aspect=1, xticks=[], yticks=[])                
                imgplot = ax[0].imshow(img1)
                ax[1].set(title=fighter2, aspect=1, xticks=[], yticks=[])
                imgplot = ax[1].imshow(img2)
            except FileNotFoundError:
                print('That picture is not in the database somehow...')
        except ValueError:
            print('Fighters have incomplete data on record... or maybe there is an issue with one of the fighters names... I honestly do not know what to do with this.')
        except TypeError:
            print('Fighters have incomplete data on record... or maybe there is an issue with one of the fighters names... I honestly do not know what to do with this.')

                
                

model score: 0.6346153846153846
cross val score: 0.6269328387484007
coefficients[[ 0.0287259   0.01037367 -0.00501167  0.74942625  0.11506916  0.02265273
   0.05862713 -0.05859741 -0.07288798 -0.07662977 -0.01612169 -0.00167001
   0.55047311  0.04833718]]
intercept[0.]


In [5]:
# We want to predict how many times out of 10 the winning fighter would win, so we look at the values
# x*theta+b. If the value is >=0 its a win and <=0 its a loss. But how far from zero gives indication of
# how likely the outcome is.
theta = winPredictionModel.coef_
b = winPredictionModel.intercept_[0]

def presigmoid_value(fighter1,fighter2,date1,date2):
    value = 0
    tup = ufc_prediction_tuple(fighter1,fighter2,date1,date2)
    for i in range(len(tup)):
        value += tup[i]*theta[0][i]
    return value + b

import math

def sigmoid(x):
    sig = 1 / (1 + math.exp(-x))
    return sig

#returns the probability that fighter1 defeats fighter2 on date1,date2
def probability(fighter1,fighter2,date1,date2):
    presig=presigmoid_value(fighter1,fighter2,date1,date2)
    return sigmoid(presig)



def manual_prediction(fighter1,fighter2,date1,date2):
    value = presigmoid_value(fighter1,fighter2,date1,date2)
    #value2 = presigmoid_value(fighter2,fighter1,date2,date1)
    return value>=0

from dateutil.relativedelta import relativedelta
def odds(fighter1,fighter2):
    date1=(date.today()- relativedelta(months=1)).strftime("%B %d, %Y")
    date2=(date.today()- relativedelta(months=1)).strftime("%B %d, %Y")
    p=probability(fighter1,fighter2,date1,date2)
    if p<.5:
        fighterOdds=round(100/p - 100)
        opponentOdds = round(1 / (1 / (1 - p) - 1) * 100)
        return ['+'+str(fighterOdds),'-'+str(opponentOdds)]
    elif p>=.5:
        fighterOdds = round(1 / (1 / p - 1) * 100)
        opponentOdds = round(100 / (1 - p) - 100)
        return ['-'+str(fighterOdds),'+'+str(opponentOdds)]
    
def give_odds(fighter1,fighter2,date1,date2):
    value = presigmoid_value(fighter1,fighter2,date1,date2)
    value2 = presigmoid_value(fighter2,fighter1,date2,date1)
    if value-value2>=0:
        winner=fighter1
    else:
        winner=fighter2
    value2 = presigmoid_value(fighter2,fighter1,date2,date1)
    abs_value = (abs(value)+abs(value2))/2
    if abs_value >=0 and abs_value <=.2:
        print(winner+" wins a little over 5 times out of 10 times.")
    elif abs_value >=.2 and abs_value <=.4:
        print(winner+" wins 6 out of 10 times.")
    elif abs_value >=.4 and abs_value <=.6:
        print(winner+" wins 7 out of 10 times.")
    elif abs_value >=.6 and abs_value <=.8:
        print(winner+" wins 9 out of 10 times.")
    elif abs_value >=.8:
        print(winner+" wins 10 out of 10 times.")
        
def money_line_odds(fighter1,fighter2,date1,date2):
    
    value = presigmoid_value(fighter1,fighter2,date1,date2)
    if value>=0:
        winner=fighter1
    else:
        winner=fighter2
    abs_value = abs(value)
    if abs_value >=0 and abs_value <=.2:
        print(winner+" wins a little over 5 times out of 10 times.")
    elif abs_value >=.2 and abs_value <=.4:
        print(winner+" wins 6 out of 10 times.")
    elif abs_value >=.4 and abs_value <=.6:
        print(winner+" wins 7 out of 10 times.")
    elif abs_value >=.6 and abs_value <=.8:
        print(winner+" wins 9 out of 10 times.")
    elif abs_value >=.8:
        print(winner+" wins 10 out of 10 times.")

In [6]:
#now we build prediction_history
#first need to rebuild "same_name" function like we did in js (done)
#next open existing prediction_history.json as a dataframe (done)
#then add all fights on the books from vegas_odds
#for each fight in prediction_history, if a prediction has not yet been made, make a prediction
#then export to json

#first call pip install python-Levenshtein
from Levenshtein import distance as lev
def same_name(str1,str2):
    str1 = str1.lower().replace('.','').replace("-",' ').replace("st.",'saint')
    str2 = str2.lower().replace('.','').replace("-",' ').replace("st.",'saint')
    str1_list=str1.split()
    str2_list=str2.split()
    str1_set=set(str1_list)
    str2_set=set(str2_list)
    if str1==str2:
        return True
    elif str1_set==str2_set:
        return True
    elif lev(str1,str2)<3:
        return True
    else:
        return False
    
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
vegas_odds=pd.read_json('src/models/buildingMLModel/data/external/vegas_odds.json')

prediction_history=pd.read_json('src/models/buildingMLModel/data/external/prediction_history.json')
#def count_null(row_number):
#    row = vegas_odds.iloc[row_number]
#    return len([i for i in range(len(row)) if row[i]==''])
#
#bookie_mask = []
#for i in range(len(vegas_odds['fighter name'])):
#    bookie_mask.append(count_null(i)<6)
#vegas_odds=vegas_odds[bookie_mask]
vegas_odds_copy=vegas_odds.copy()
for col in ['predicted fighter odds','predicted opponent odds','average bookie odds','correct?']:
    vegas_odds_copy[col]=""

In [ ]:
vegas_odds_copy

In [ ]:
#use this to fix any names in the dataframe
#vegas_odds_copy.at[7, 'opponent name'] = 'Marcos Rogerio de lima'
#vegas_odds_copy

In [ ]:
#filling in predictions
for i in vegas_odds_copy.index:
    fighter=vegas_odds_copy['fighter name'][i]
    opponent=vegas_odds_copy['opponent name'][i]
    if in_ufc(fighter) and in_ufc(opponent):
        odds_calc = odds(fighter,opponent)
        print('predicting: '+fighter,'versus '+opponent,'.... '+str(odds_calc))
        vegas_odds_copy['predicted fighter odds'][i]=odds_calc[0]
        vegas_odds_copy['predicted opponent odds'][i]=odds_calc[1]
        sum_av_f=0
        tot_av_f=0
        sum_av_o=0
        tot_av_o=0
        for bookie in ['DraftKings', 'BetMGM', 'Caesars', 'BetRivers', 'FanDuel', 'PointsBet', 'Unibet', 'Bet365', 'BetWay', '5D', 'Ref']:
            if vegas_odds_copy['fighter '+bookie][i]!='':
                sum_av_f+=int(vegas_odds_copy['fighter '+bookie][i])
                tot_av_f+=1
            if vegas_odds_copy['opponent '+bookie][i]!='':
                sum_av_o+=int(vegas_odds_copy['opponent '+bookie][i])
                tot_av_o+=1
        vegas_odds_copy['average bookie odds'][i]=[str(round(sum_av_f/tot_av_f)),str(round(sum_av_o/tot_av_o))]



In [ ]:
vegas_odds_copy

In [ ]:
prediction_history

In [ ]:
prediction_history = pd.concat([vegas_odds_copy, prediction_history], axis = 0).reset_index(drop=True)
prediction_history

In [ ]:
prediction_history.drop_duplicates(subset =["fighter name", "opponent name"],
                     keep = 'first', inplace = True)
prediction_history

In [ ]:
import json
result = prediction_history.to_json()
parsed = json.loads(result)
jsonFilePath='src/models/buildingMLModel/data/external/prediction_history.json'
with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
    jsonf.write(json.dumps(parsed, indent=4))
print('saved to '+jsonFilePath)

In [ ]:
#a couple examples

In [ ]:
ufc_prediction_tuple('Blagoy Ivanov', 'Cameron Else', 'March 26, 2022','March 26, 2022')

In [ ]:
odds('Joel Alvarez', 'Calvin Kattar')

In [ ]:
#HERE WE START SOME UNNECESSARY STUFF THAT SHOULDN"T RUN

In [ ]:
# ----------- SKIP --------------- #

In [ ]:
#NN with a pipeline
#this is overfit because it fixes a specific random state for test_train_split... you have to average over a list
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
scores=[]
for i in range(5,20):
    for j in range(5,20):
        for k in range(1,20):
            X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
            clf = MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(i,j), random_state=k, max_iter=2000)
            pipe = make_pipeline(StandardScaler(), clf)
            pipe.fit(X_train, y_train)  # apply scaling on training data
            score=pipe.score(X_test, y_test)
            scores.append([i,j,k,score])

In [ ]:
#the highest scoring NN (overfit... see below) (change the random state from 42 and observe)
max([scores[j][3] for j in range(len(scores))])

In [ ]:
#Here is the highest scoring NN model (almost 67%!!!)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
clf = MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(9,13), random_state=11, max_iter=2000)
pipe = make_pipeline(StandardScaler(), clf)
pipe.fit(X_train, y_train)  # apply scaling on training data
print('Accuracy: '+str(pipe.score(X_test, y_test)))

In [ ]:
#but we need to further cross validate this (even though it was trained on a test set...) just to be safe

In [ ]:
#lets take the average over all test_train_split random_states (decrease to .623)
clf = MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(9,13), random_state=11, max_iter=2000)
pipe = make_pipeline(StandardScaler(), clf)
scores=[]
for i in range(1,100):
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=i)
    pipe.fit(X_train, y_train)  # apply scaling on training data
    scores.append(pipe.score(X_test, y_test))
print('Average Score: '+str(sum(scores)/len(scores)))

In [ ]:
#cross validated NN with a standardscalar-clf pipeline (current best is .636)
#THIS DOES NOT OUTPERFORM LR!!!
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
avscores=[]
for i in range(5,30):
    for j in range(2,20):
        for k in range(1,5):
            scores=[]
            for l in range(1,4):
                X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=l)
                clf = MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(i,j), random_state=k, max_iter=2000)
                pipe = make_pipeline(StandardScaler(), clf)
                pipe.fit(X_train, y_train)  # apply scaling on training data
                score=pipe.score(X_test, y_test)
                scores.append(score)
            avscores.append([i,j,k,sum(scores)/len(scores)])
                
                
                

In [ ]:
maxav=max([avscores[j][3] for j in range(len(avscores))]);maxav

In [ ]:
topscores=[j for j in range(len(avscores)) if avscores[j][3]>maxav-.005];topscores

In [ ]:
for i in range(len(topscores)):
    print(avscores[topscores[i]])

In [ ]:
#trying random forest for winner prediction
#not bad.. may need to redo feature analysis specifically for this
#also try messing with parameters
ufc_fights_df = ufc_fights_winner[current_best]
X=ufc_fights_df.iloc[0:2080].to_numpy()
y=ufc_fights_winner['result'].iloc[0:2080]
rfc=RandomForestClassifier()
X_train, X_test, y_train, y_test = train_test_split(X, y)
rfc=RandomForestClassifier()
pipe = make_pipeline(StandardScaler(), rfc)
pipe.fit(X_train, y_train)  # apply scaling on training data
pipe.score(X_test,y_test)

In [ ]:
#Lets try a NN on the entire dataset (does very poorly...)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
X=ufc_fights_winner[possible_stats].iloc[0:40*55].to_numpy()
y=ufc_fights_winner['result'].iloc[0:40*55]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
clf = MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(len(possible_stats),len(possible_stats),), random_state=11, max_iter=5000)
pipe = make_pipeline(StandardScaler(), clf)
pipe.fit(X_train, y_train)  # apply scaling on training data
print('Accuracy: '+str(pipe.score(X_test, y_test)))

In [ ]:
#random forest does way better for picking the method!
best_stats=['fighter_ko_losses_diff_2', 'fighter_L2Y_sub_losses_diff_2']
rfc=RandomForestClassifier()
Xr=ufc_fights_method[best_stats].iloc[0:2300]
yr=ufc_fights_method['method'].iloc[0:2300]

rfc.fit(Xr,yr)
accuracy = cross_val_score(rfc,Xr,yr,cv=3).mean()
precision = cross_val_score(rfc,Xr,yr,cv=3, scoring='precision_micro').mean()
recall = cross_val_score(rfc, Xr, yr, cv=3, scoring='recall_macro').mean()
print('Accuracy: '+str(accuracy),'F1 score: '+str(precision*recall/(precision+recall)))

In [ ]:
# doing a one time thing with prediction history

In [ ]:
prediction_history

In [ ]:
#below we are penciling in first_event I did on paper (not part of pipeline, just a one time thing)

In [ ]:
first_event = prediction_history.copy()
#filled in some things by hand but later deleted that code
first_event

In [ ]:
#filling in predictions 
#need to make sure it only makes predictions for fights it hasn't yet predicted
for i in first_event.index:
    fighter=first_event['fighter name'][i]
    opponent=first_event['opponent name'][i]
    odds_calc = odds(fighter,opponent)
    first_event['predicted fighter odds'][i]=odds_calc[0]
    first_event['predicted opponent odds'][i]=odds_calc[1]
    sum_av_f=0
    tot_av_f=0
    sum_av_o=0
    tot_av_o=0
    for bookie in ['DraftKings', 'BetMGM', 'Caesars', 'BetRivers', 'FanDuel', 'PointsBet', 'Unibet', 'Bet365', 'BetWay', '5D', 'Ref']:
        if first_event['fighter '+bookie][i]!='':
            sum_av_f+=int(first_event['fighter '+bookie][i])
            tot_av_f+=1
        if first_event['opponent '+bookie][i]!='':
            sum_av_o+=int(first_event['opponent '+bookie][i])
            tot_av_o+=1
    first_event['average bookie odds'][i]=[str(round(sum_av_f/tot_av_f)),str(round(sum_av_o/tot_av_o))]
first_event



In [ ]:
import json
result = first_event.to_json()
parsed = json.loads(result)
jsonFilePath='src/models/buildingMLModel/data/external/prediction_history.json'
with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
    jsonf.write(json.dumps(parsed, indent=4))
print('saved to '+jsonFilePath)

In [ ]:
#values from javascript
#comparing to make sure we are making the same predictions on the website as we are here
tup2=[0.00,3.00,3.00,-1.00,1.00,-2,-1,-1,-4,0.91,1.40,0.40,-0.29,1.95]
coeff2=[0.03,0.01,-0.01,0.69,0.13,0.03,0.06,-0.05,-0.07,-0.07,-0.02,-0.01,0.48,0.05]
for i in range(len(tup2)):
    print(tup2[i],coeff2[i])

In [ ]:
#need this for standard deviation used in odds
#building the set of presigmoid values for all fights in ufc_fights_df
X=ufc_fights_df.iloc[0:40*75].to_numpy()
prediction_values=[]
for i in range(40*75):
    value = 0
    tup = X[i]
    for i in range(len(tup)):
        value += tup[i]*theta[0][i]
    prediction_values.append(value + b)

In [ ]:
#shows the range of 
plt.hist(prediction_values)

In [ ]:
std=np.std(prediction_values);std

In [ ]:
for stat in ufc_fights.columns:
    print(stat)

In [ ]:
#checking by hand that the accuracy of the manual predictions are the same as the accuracy 
#getting .61 after 100 iterations but .56 after 200 iterations (need longer run)
#.53 after 400 iterations (need longer run)
#.545 for 100 iterations (starting at number 400 so [400:500])
#.62 for 500 iterations (starting at number 500 so [500:1000])
#.607 for 1000 iterations (starting at number 1000 so [1000:2000])
# .58 for 1000 iterations (starting at number 2000 so [2000:3000])


#given by python's model score (around .61 at time of writing... Mar 18 2022)
#takes forever... about 3 seconds per computation
real_values = ufc_fights['result']
is_model_correct=[]
index=0
for i in ufc_fights.index[:10]:
    try:
        index+=1
        f1 = ufc_fights['fighter'][i]
        f2 = ufc_fights['opponent'][i]
        date = ufc_fights['date'][i]
        if (manual_prediction(f1,f2,date,date)):
            my_pred = 'W'
        else:
            my_pred = 'L'
        print(f1,f2,my_pred,str(real_values[i]))
        if (my_pred==real_values[i]):
            is_model_correct.append(1)
        else:
            is_model_correct.append(0)
    except:
        continue
    if index>0 and index%10==0:
        manual_accuracy = sum(is_model_correct)/len(is_model_correct)
        print('The accuracy of the model is computed as: '+str(manual_accuracy)+' after '+str(index)+' iterations')
        
        

In [ ]:
#making sure the manual model predictions are as expected
#takes forever... about 3 seconds per computation
real_values = ufc_fights['result']
is_model_correct=[]

for i in ufc_fights.index[:10]:
    try:
        f1 = ufc_fights['fighter'][i]
        f2 = ufc_fights['opponent'][i]
        date = ufc_fights['date'][i]
        ufc_predict(f1,f2,date,date)
    except:
        print('this fight didnt compute right')

In [ ]:
manual_prediction('Thiago Santos','Magomed Ankalaev', 'March 12, 2022', 'March 12, 2022')

In [ ]:
probability('Belal Muhammad','Vicente Luque','April 9, 2022','April 9, 2022')

In [ ]:
ufc_predict('Magomed Ankalaev', 'Thiago Santos','March 12, 2022', 'March 12, 2022')

In [ ]:
#checking the distribution of the average absolute values of presigmoid values
#this tells us the thresholds for approximating odds (wins out of 10)
#this takes super long... about 3 seconds per computation
#found best bins to be 0, .2, .4, .6, .8, 1, etc going as 5 6 7 8 9 10 times out of 10
prediction_average_absolute_values=[]
for i in ufc_fights_winner.index[:10]:
    try:
        f1 = ufc_fights['fighter'][i]
        f2 = ufc_fights['opponent'][i]
        date = ufc_fights['date'][i]
        value1 = presigmoid_value(f1,f2,date,date)
        value2 = presigmoid_value(f2,f1,date,date)
        prediction_average_absolute_values.append((abs(value1)+abs(value2))/2)
    except:
        continue

In [ ]:
bins_for_absavg=[]
for i in range(35):
    bins_for_absavg.append(i/25)
plt.hist(prediction_average_absolute_values, bins=bins_for_absavg)

In [ ]:
winPredictionModel.intercept_[0]

In [ ]:
presigmoid_value('Jorge Masvidal','Colby Covington', 'March 5, 2022', 'March 5, 2022')

In [ ]:
ufc_prediction_tuple('Khabib Nurmagomedov','Colby Covington', 'March 1, 2022', 'March 1, 2022')

In [ ]:
give_odds('Colby Covington','Khabib Nurmagomedov', 'March 5, 2022', 'March 5, 2022')

In [ ]:
money_line_odds('Colby Covington','Khabib Nurmagomedov', 'March 5, 2022', 'March 5, 2022')

In [ ]:
#NEW IDEA. PERHAPS FOR EACH FIGHT, WE LOOK AT THE CORRESPONDING POINT IN THE SPACE OF ALL FEATURES, AND THEN
#BUILD A MODEL WHERE THE TRAINING SET ONLY INVOLVES POINTS NEAR THAT POINT.
#THIS MIGHT BE MORE EFFECTIVE BECAUSE THE MOST RELEVANT FEATURES IS LIKELY TO DEPEND ON THE LOCATION
#IN THE PARAMETER SPACE.